# A Developer's Guide to the AIOS Metrics System

Welcome to this guide on the OpenOS/AIGr.id Metrics System. This notebook serves as a standalone tutorial that walks you through every step of monitoring and interpreting metrics for your AIOS Blocks. We will explore how to access live metrics, understand their structure, and leverage them for monitoring, debugging, and autoscaling.

## The AIOS Metrics System: A Brief Overview

The AIOS Metrics System is a crucial component of the AIOS ecosystem, providing real-time insights into the health, performance, and status of all running Blocks. It is designed to be a comprehensive, scalable, and extensible monitoring solution that enables developers to:

- **Monitor Block Health**: Keep track of uptime, resource utilization (CPU, GPU, memory), and other key health indicators.
- **Analyze Performance**: Measure inference latency, throughput (tokens per second), and other performance-related metrics.
- **Debug Issues**: Identify bottlenecks, errors, and anomalies by inspecting detailed metrics from each instance.
- **Drive Autoscaling**: Use metrics to define and trigger autoscaling policies, ensuring that your applications can handle varying loads efficiently.

This guide will cover the essential aspects of the Metrics System, demonstrating how to query metrics, interpret the results, and use them to manage your Blocks effectively.


## Metrics Tutorial Overview

In this guide, we will cover the following steps in detail:

1. **Accessing Block Metrics**: We will use `curl` to fetch live metrics from a running Block.
2. **Understanding the Metrics Structure**: We will break down the JSON response to understand the different types of metrics available.
3. **Interpreting Key Metrics**: We will explore the meaning and significance of important metrics, such as latency, throughput, and resource utilization.
4. **Visualizing Metrics**: We will use Python libraries to plot and visualize metrics data, making it easier to identify trends and patterns.
5. **Custom Metrics**: We will briefly touch upon how to implement and expose custom metrics from your own Blocks.

Let's get started!


## 1. Accessing Block Metrics

The primary way to access metrics for a running Block is through the Metrics API, which exposes a simple HTTP endpoint. You can use a tool like `curl` to send a GET request to this endpoint and retrieve the latest metrics data in JSON format.

The Metrics API endpoint is typically structured as follows:

`http://<metrics-service-ip>:<port>/block/<block-id>`

For this tutorial, we will use the following live metrics URL for the `gemma3-27b-block`:

`http://MANAGEMENTMASTER:30201/block/gemma3-27b-block`

Let's use `curl` to fetch the metrics for this Block.


In [ ]:
import requests
import json

# Define the metrics URL
metrics_url = "http://MANAGEMENTMASTER:30201/block/gemma3-27b-block"

# Fetch the metrics data
try:
    response = requests.get(metrics_url)
    response.raise_for_status()  # Raise an exception for bad status codes
    metrics_data = response.json()
    
    # Pretty-print the JSON response
    print(json.dumps(metrics_data, indent=2))
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching metrics: {e}")
    # As a fallback, load the sample metrics data from the local file
    print("\\nLoading sample metrics from local file...")
    with open('/home/srikanth_g_cognitif_ai/files_from_local/documentation/video_tutorial_series/pre_and_post_processing_metrics_streaming_health/sample_metrics.json', 'r') as f:
        metrics_data = json.load(f)
        print(json.dumps(metrics_data, indent=2))


## 2. Understanding the Metrics Structure

The metrics data is returned as a JSON object with a well-defined structure. Let's break down the key components of the response:

- **`data`**: The root object containing an array of Block metrics.
- **`blockId`**: The unique identifier for the Block.
- **`instances`**: An array of metric objects, one for each running instance of the Block. This includes the main `executor` and each of the worker instances (e.g., `in-vkkv`).

Each instance object contains a rich set of metrics, which can be broadly categorized as follows:

- **System Metrics**: Basic health and status information, such as `uptime`, `uptime_hours`, and `uptime_minutes`.
- **Hardware Metrics**: Resource utilization data for the underlying hardware, including `cpu`, `gpus`, and `memory`.
- **LLM-Specific Metrics**: A detailed set of metrics related to the performance of the Language Model, such as `llm_tokens_per_second`, `llm_time_to_first_token_seconds_sum`, and `llm_prompt_tokens_total`.
- **Rolling Averages**: Many metrics are also available as rolling averages over different time periods (e.g., `average_1m`, `average_5m`, `average_15m`), which are useful for smoothing out short-term fluctuations and identifying longer-term trends.


## 3. Interpreting Key Metrics

Let's take a closer look at some of the most important metrics and what they tell us about the Block's performance and health:

- **`llm_tokens_per_second`**: This metric measures the throughput of the model, indicating how many tokens it can generate per second. A higher value generally means better performance.
- **`llm_time_to_first_token_seconds_sum`**: This is the total time it takes for the model to generate the first token after receiving a prompt. A lower value is desirable, as it indicates a more responsive model.
- **`llm_active_sessions`**: This metric shows the number of active inference sessions currently being handled by the instance. It's a good indicator of the instance's current load.
- **`hardware.gpus[].utilization`**: This shows the utilization of each GPU in percentage. High utilization might indicate that the model is making good use of the available hardware, but it could also be a sign of a bottleneck if it's consistently at 100%.
- **`hardware.memory.averageUtil`**: This is the average memory utilization of the instance. A high value could indicate a memory leak or that the model requires more memory than allocated.
- **`end_to_end_latency`**: This measures the total time from when a request is sent to when the final response is received. It's a crucial metric for user-facing applications where responsiveness is key.


## 4. Visualizing Metrics

While raw JSON data is useful, visualizing metrics can make it much easier to spot trends, anomalies, and patterns. In this section, we'll use Python's `matplotlib` and `pandas` libraries to create some basic plots of our metrics data.

First, let's extract the relevant data from our `metrics_data` object and load it into a pandas DataFrame.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Extract the instance metrics from the JSON data
instance_metrics = metrics_data.get('data', [{}])[0].get('instances', [])

# Create a list to store the processed data
data_for_df = []

# Process each instance's metrics
for instance in instance_metrics:
    if instance.get('type') == 'app':  # We are interested in the application instances
        instance_id = instance.get('instanceId')
        llm_tps = instance.get('llm_tokens_per_second', 0)
        gpu_util = instance.get('hardware', {}).get('gpus', [{}])[0].get('utilization', 0)
        cpu_util = instance.get('hardware', {}).get('cpu', {}).get('percent', 0)
        mem_util = instance.get('hardware', {}).get('memory', {}).get('averageUtil', 0)
        
        data_for_df.append({
            'Instance ID': instance_id,
            'LLM Tokens/sec': llm_tps,
            'GPU Utilization (%)': gpu_util,
            'CPU Utilization (%)': cpu_util,
            'Memory Utilization (%)': mem_util * 100  # Convert to percentage
        })

# Create a pandas DataFrame
df = pd.DataFrame(data_for_df)

# Plot the data
if not df.empty:
    df.set_index('Instance ID', inplace=True)
    
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))
    fig.suptitle('Block Performance Metrics', fontsize=16)
    
    df['LLM Tokens/sec'].plot(kind='bar', ax=axes[0,0], title='LLM Throughput')
    axes[0,0].set_ylabel('Tokens/sec')
    
    df['GPU Utilization (%)'].plot(kind='bar', ax=axes[0,1], title='GPU Utilization')
    axes[0,1].set_ylabel('%')
    
    df['CPU Utilization (%)'].plot(kind='bar', ax=axes[1,0], title='CPU Utilization')
    axes[1,0].set_ylabel('%')
    
    df['Memory Utilization (%)'].plot(kind='bar', ax=axes[1,1], title='Memory Utilization')
    axes[1,1].set_ylabel('%')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
else:
    print("No application instance metrics found to plot.")


## 5. Custom Metrics & `register_custom_metrics`

The AIOS Metrics System is extensible, allowing you to define and expose your own custom metrics from your Blocks. This is particularly useful when you have application-specific metrics that you want to monitor, such as the number of active users, the cache hit rate, or the number of times a specific feature is used.

### How to Implement Custom Metrics

Implementing a custom metric is straightforward with the `aios_instance` SDK. Here is the general process:

1.  **Identify the Metric**: First, decide what application-specific value you want to monitor. For example, let's say you want to track a "user satisfaction score" that is provided after each inference.

2.  **Instrument Your Code**: In your Block's code, where the value is generated (e.g., in the function that receives user feedback), you will call the `observe_rolling()` or `observe_custom_rolling()` method on your `AIOSMetrics` instance.

3.  **Implicit Registration**: You don't need to explicitly call a `register_custom_metrics` function. The first time you call `observe_rolling("your_custom_metric_name", value)`, the metric is automatically registered within the `AIOSMetrics` object.

4.  **View the Metric**: Once your code is running, the new custom metric will automatically appear in the main metrics JSON output, complete with its `current` value and rolling averages (`average_1m`, `average_5m`, `average_15m`). From there, it can be scraped by monitoring systems or used in autoscaling policies.

The `metrics_updated.py` file provides an excellent example of how to extend the `LLMMetrics` class to add custom metrics and integrate with the native timing capabilities of the `llama_cpp` library. Let's look at a more detailed conceptual example below.

For more detailed information on how to implement custom metrics, please refer to the official documentation:

- [Custom Metrics in AIOS](https://github.com/OpenCyberspace/OpenOS.AI-Documentation/blob/main/metrics-system/metrics-system.md#custom-metrics)


## AIOS Metrics SDKs & Rolling Metrics

The AIOS platform provides a set of SDKs to simplify the process of instrumenting your applications and collecting metrics. Two key SDKs are:

- **`aios_instance` SDK**: This is a general-purpose SDK that provides the core functionality for registering and collecting metrics. It includes the `AIOSMetrics` class, which handles the registration of Prometheus metrics, the collection of hardware metrics, and the aggregation of **rolling metrics**. It is designed to be used by any type of Block, regardless of the underlying framework or application.

- **`aios_llama_cpp` SDK**: This is a specialized SDK that is designed to be used with Blocks that are based on the `llama_cpp` framework. It extends the functionality of the `aios_instance` SDK by providing a set of pre-defined metrics that are specific to LLM Blocks. It also includes the `LLMMetrics` class, which simplifies the process of collecting and reporting LLM-specific metrics, such as tokens per second, time to first token, and prompt/generation token counts.

### What are Rolling Metrics?

A **rolling metric** is a time-windowed average of a metric's values. Instead of just showing the current value, it provides averages over predefined periods (e.g., 1, 5, and 15 minutes). This is crucial for several reasons:

- **Smoothing**: It smooths out short-term spikes and dips, providing a more stable and representative view of the Block's performance over time.
- **Trend Analysis**: It helps in identifying longer-term trends. For example, a gradually increasing rolling average for latency might indicate a performance degradation issue.
- **Intelligent Autoscaling**: Autoscaling policies rely on rolling metrics to make informed decisions. Scaling based on an instantaneous value can lead to erratic behavior (e.g., scaling up for a 1-second spike and then immediately scaling down). Rolling averages ensure that scaling decisions are based on sustained load patterns.

The `aios_instance` SDK's `AIOSMetrics` class handles the complexity of calculating these rolling averages for you. When you use the `observe_rolling()` method, the SDK adds the value to a time-series data structure, and the rolling averages are automatically computed and exposed in the metrics endpoint.

The `metrics_updated.py` file provides an excellent example of how to extend the `LLMMetrics` class to add custom metrics and integrate with the native timing capabilities of the `llama_cpp` library.


In [ ]:
# This is a conceptual example based on metrics_updated.py to illustrate how you might
# implement and use custom metrics. This code is not meant to be run directly in this notebook,
# but rather to serve as a reference for your own Block implementations.

import time
import logging
from collections import deque

# --- This would typically be in the aios_instance SDK ---
class RollingMetric:
    """A helper class to manage time-windowed data for rolling averages."""
    def __init__(self, window_seconds=900):
        self.window = window_seconds
        self.data = deque()

    def add(self, value):
        now = time.time()
        self.data.append((now, value))
        self._cleanup(now)

    def _cleanup(self, now):
        while self.data and self.data[0][0] < now - self.window:
            self.data.popleft()

    def average(self, window_seconds):
        now = time.time()
        self._cleanup(now)
        values = [v for t, v in self.data if t >= now - window_seconds]
        return sum(values) / len(values) if values else 0

class AIOSMetrics:
    """A simplified version of the AIOSMetrics class for demonstration."""
    def __init__(self):
        self.rolling_metrics = {}
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def observe_rolling(self, name, value):
        """Observes a value for a rolling metric. Implicitly registers the metric on first call."""
        if name not in self.rolling_metrics:
            self.logger.info(f"Implicitly registering new rolling metric: '{name}'")
            self.rolling_metrics[name] = RollingMetric()
        self.rolling_metrics[name].add(value)
        self.logger.info(f"Observed value for '{name}': {value}")

# --- This would be your custom metrics logic within your Block's code ---
class MyBlockMetrics:
    def __init__(self, metrics_backend: AIOSMetrics):
        self.metrics = metrics_backend
        self.logger = logging.getLogger(__name__)

    def log_user_feedback(self, score: float):
        """Logs a custom metric for user satisfaction."""
        # This call will create the 'user_satisfaction_score' metric if it doesn't exist,
        # and then add the new score to its time-windowed data queue.
        self.metrics.observe_rolling("user_satisfaction_score", score)

    def log_cache_hit_ratio(self, ratio: float):
        """Logs another custom metric, for example, a cache hit ratio."""
        if 0.0 <= ratio <= 1.0:
            self.metrics.observe_rolling("cache_hit_ratio", ratio)
        else:
            self.logger.warning(f"Invalid cache hit ratio: {ratio}. Must be between 0.0 and 1.0.")


# --- Example Usage (conceptual simulation) ---
print("--- Simulating Custom Metrics ---")
# 1. In your Block, you would initialize the AIOSMetrics backend.
aios_metrics_backend = AIOSMetrics()

# 2. You would then create your custom metrics handler.
my_block_metrics = MyBlockMetrics(aios_metrics_backend)

# 3. As your Block runs, it calls the logging methods.
print("\\n--- Logging user feedback scores ---")
my_block_metrics.log_user_feedback(4.5) # User gives a rating of 4.5
time.sleep(0.1) # Simulate time passing
my_block_metrics.log_user_feedback(3.0) # Another user gives a rating of 3.0
time.sleep(0.1)
my_block_metrics.log_user_feedback(5.0)

print("\\n--- Logging cache hit ratios ---")
my_block_metrics.log_cache_hit_ratio(0.85)
time.sleep(0.1)
my_block_metrics.log_cache_hit_ratio(0.92)

# 4. The AIOS Metrics System would then expose this data. Here's how to see the result:
print("\\n--- Retrieving simulated metrics ---")
satisfaction_metric = aios_metrics_backend.rolling_metrics.get("user_satisfaction_score")
if satisfaction_metric:
    # The average over the last 60 seconds (in this simulation, it includes all values)
    avg_satisfaction = satisfaction_metric.average(60)
    print(f"Average User Satisfaction (1m): {avg_satisfaction:.2f}")

cache_metric = aios_metrics_backend.rolling_metrics.get("cache_hit_ratio")
if cache_metric:
    avg_cache_ratio = cache_metric.average(60)
    print(f"Average Cache Hit Ratio (1m): {avg_cache_ratio:.2f}")

# Example of what the final JSON might contain for these custom metrics:
# "user_satisfaction_score": {
#   "current": 5.0,
#   "average_1m": 4.17,
#   ...
# },
# "cache_hit_ratio": {
#   "current": 0.92,
#   "average_1m": 0.89,
#   ...
# }


## Conclusion

In this tutorial, we have explored the AIOS Metrics System, learning how to access, interpret, and visualize metrics for a running Block. We have seen how the Metrics System provides a comprehensive set of tools for monitoring the health, performance, and status of your AI applications.

By leveraging the insights provided by the Metrics System, you can ensure that your applications are running smoothly, identify and resolve issues quickly, and make informed decisions about how to scale and optimize your resources.

We encourage you to explore the Metrics System further and to integrate it into your own AIOS development workflow.
